In [1]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np
import matplotlib.pyplot as plt
from consav.linear_interp import interp_4d

# load modules related to this project
from educationmodel import EducationModel
import estimate as est 

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [2]:
model = EducationModel() # Define the model

In [ ]:
model.solve() # Solve the model

# Redefine
sol = model.sol 
par = model.par 


## Simulate Model

In [ ]:
# Simulate model
model.simulate()
sim=model.sim

In [ ]:
# Determining the different types
Type1 = np.where((sim.util_school_fix == 50) & (sim.abil_work_fix == 30),1,0) # Low utility school, low ability work
Type2 = np.where((sim.util_school_fix == 200) & (sim.abil_work_fix == 30),1,0) # High utility school, low ability work
Type3 = np.where((sim.util_school_fix == 50) & (sim.abil_work_fix == 120),1,0) # Low utility school, high ability work
Type4 = np.where((sim.util_school_fix == 200) & (sim.abil_work_fix == 120),1,0) # High utility school, high ability work


In [ ]:
mean_school_vectors =[]
mean_interrupt_vectors =[]
mean_school_time_vectors = []
mean_wage_vectors = []
mean_experience_vectors = []

In [ ]:
types = [Type1,Type2,Type3,Type4]
for s in types:
    i = np.where(s == 1)[0]

    mean_school = sim.school[i].mean(axis=0)
    mean_interrupt = sim.interrupt[i].mean(axis=0)
    mean_school_time = sim.school_time[i].mean(axis=0)
    mean_wage = np.nanmean(sim.wage[i],axis=0)
    mean_experience = np.nanmean(sim.experience[i],axis=0)
    mean_school_vectors.append(mean_school)
    mean_interrupt_vectors.append(mean_interrupt)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience_vectors.append(mean_experience)


In [ ]:
plot_vectors = [mean_school_vectors, mean_interrupt_vectors, mean_school_time_vectors, mean_wage_vectors, mean_experience_vectors]
vector_names = ["Mean schooling choice","Mean interruption choice", "Mean school time", "Mean wage", "Mean experience"]

In [ ]:
for vector, name in zip(plot_vectors, vector_names):
    for i in range(4):
        plt.plot(vector[i], label=f"{name} for type {i+1}")
    plt.legend()
    plt.xlabel("Time period")
    plt.ylabel(name)
    plt.savefig(f"{name}_for_type_{i+1}_over_time.png") # Save the figure
    plt.show()


## Estimate the parameters

In [3]:
estimate_ = est.estimate_class() # define the estimate function
family_data, decision_data = estimate_.read_data() # Read data

In [ ]:
print(f'Max util_school: {util_school.max()}, Min util_school: {util_school.min()}')
print(f'Max ability_wage: {ability_wage.max()}, Min ability_wage: {ability_wage.min()}')

In [ ]:
delta7= np.linspace(-5,5,5)
log_lik = np.nan + np.zeros(5)

In [ ]:
for i in range(5):
    pnames = ['delta7']
    theta = np.array([delta7[i]])
    log_lik[i]=estimate_.ll(theta,model, family_data, decision_data, pnames)

In [ ]:
fig = plt.figure(figsize=(6,4),dpi=100)

ax = fig.add_subplot(1,1,1)
ax.set_title('Log likelihood')

ax.plot(delta7,log_lik)

ax.set_xlabel('delta7')
ax.set_ylabel('log likelihood')


fig.tight_layout()

In [ ]:
pnames = ['delta7', 'delta8']
theta0 = [-0.8883146062919914]

In [4]:
pnames = ['delta7','delta8']
theta0 = [-1.0979559481181265,-0.0001]


In [8]:
res = estimate_.estimate(model, family_data, decision_data, pnames, theta0)

delta7 -1.0979559481181265
delta7 -1.0979559381181265


KeyboardInterrupt: 

In [ ]:
cov, se = estimate_.variance(model,['delta7'],family_data,decision_data,np.array([-1.0979559481182595]))

In [ ]:
pnames = 'delta7'

In [ ]:
# delta 7 = -0.8883146062919914

In [ ]:
thetahat = np.array([-0.883146062919914])
thetahat

In [ ]:
thetahat.ndim

In [ ]:
cov, se = estimate_.variance(model,pnames,family_data,decision_data,thetahat)

In [ ]:
fq = lambda theta: estimate_.obj(theta,model,family_data,decision_data,pnames)

In [ ]:
fq

In [ ]:
thetahat

In [ ]:
thetahat.ndim

In [ ]:
thetahat

In [ ]:
estimate_.hessian(f_q)

In [ ]:
f_q = lambda theta: estimate_.obj(theta,model,family_data,decision_data,pnames)

In [ ]:
x0 = np.array([0.4])

In [ ]:
cov, se = estimate_.variance(model,pnames,family_data,decision_data,x0)

In [ ]:
se

In [ ]:
estimate_.hessian(x0,f_q)

In [ ]:
f_q([0.4])

### Illustrate results

In [ ]:
# Find the likelihood value for different combinations 
par_RC = copy.copy(model.RC)
par_c = copy.copy(model.c)

# Prøver for forskellige værdier af RC og c. 
Ndelta7 = 20
Ndelta8 = 20

log_lik = np.nan + np.zeros((NRC,Nc))
RC= np.linspace(8,15,NRC)
c = np.linspace(1,4,Nc)

for i in range(NRC):
    for j in range(Nc):
        # Estimate delta7 and delta8 
        pnames = ['RC','c']
        theta = np.array([RC[i], c[j]])
        log_lik[i,j]= estimate_.ll(theta, model, family_data,decision_data,pnames)

#Re-inset the true parameters 
model.RC = copy.copy(par_RC)
model.c = copy.copy(par_c)

In [ ]:
# plot figure in three dimensions
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.style.use('seaborn-whitegrid')


fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1,projection='3d', computed_zorder=False)

# Make data.
X, Y = np.meshgrid(RC, c,indexing='ij')
x, y = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# Plot the surface.
surf = ax.plot_surface(X, Y,  , cmap=cm.jet)

#Plot max value
max = ax.scatter(RC[x], c[y], log_lik[x,y], color=['black'], marker='o', s=10)

# Customize the axis.
ax.set_xlabel(f'RC')
ax.set_ylabel(f'c')
ax.set_title(f'Log-likelihood (RC,c)')
ax.invert_xaxis()

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()